## Selección automática de la mejor técnica para sustitución con Sklearn

En este notebook vamos a hacer búsqueda exhaustiva de hiper-parámetros o búsqueda de cuadricula por su nombre en inglés 'grid search' sobre los métodos de sustitución disponibles en Scikit-learn para determinar cuáles técnicas de imputación funcionan mejor para este conjunto de datos y el modelo de aprendizaje de máquina que seleccionemos

También vamos a entrenar un modelo simple de machine learning como parte de un flujo de trabajo o 'pipeline'.


## En este demo:

Vamos a aprender a **seleccionar automáticamente la mejor técnica de sustitución utilizando Scikit-learn** y los datos Ames House Price.

- Para bajar los datos, por favor referirse a la clase **Datasets** en la  **Sección 1** del curso.


In [1]:
import pandas as pd
import numpy as np

# importar clases para sustitución
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# importar extra clases para modelamiento
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0)

In [2]:
# carguemos los datos 
data = pd.read_csv('../houseprice.csv')
print(data.shape)
data.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# encontrar las variables categóricas
# aquellas de tipo 'Object'  en los datos

features_categorical = [c for c in data.columns if data[c].dtypes=='O']

# encontrar las variables numéricas
# aquellas que no son 'Object' excluyendo también el target SalePrice

features_numerical = [c for c in data.columns if data[c].dtypes!='O' and c !='SalePrice']

In [4]:
# revisemos las variables categóricas

data[features_categorical].head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [5]:
# revisemos las variables numéricas

data[features_numerical].head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,836,192,84,0,0,0,0,0,12,2008


In [6]:
# separar datos en segmentos entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(
    data.drop('SalePrice', axis=1),  # removemos el target del set de datos
    data['SalePrice'],  # el target
    test_size=0.3,  
    random_state=0)  

X_train.shape, X_test.shape

((1022, 80), (438, 80))

In [7]:
# Creemos el flujo de trabajo o tubería de pre-procesamiento (preprocessing pipelines) para las variables categóricas y numéricas

# adaptado de  Scikit-learn: código disponible aquí bajo la licencia BSD3 :
# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numeric_transformer, features_numerical),
        ('categorical', categorical_transformer, features_categorical)])

# Para iniciar el pipeline es necesario pasar un argumento a los transformadores
# Estos serán modificados durante la búsqueda de cuadricula gridsearch a continuación.

In [8]:
# Anadir un clasificador al preprocessing pipeline.
# ahora tenemos un flujo de trabajo (pipeline) completo para predicción.

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', Lasso(max_iter=2000))])

In [9]:
# creemos la cuadrícula con todos los parámetros que queremos probar

param_grid = {
    'preprocessor__numerical__imputer__strategy': ['mean', 'median'],
    'preprocessor__categorical__imputer__strategy': ['most_frequent', 'constant'],
    'classifier__alpha': [10, 100, 200],
}

grid_search = GridSearchCV(clf, param_grid, cv=5, iid=False, n_jobs=-1, scoring='r2')

# cv=3 es la validación cruzada (cross-validation)
# no_jobs =-1 usar todas las CPUs disponibles  
# scoring='r2' indica que se evalué usando r cuadrado

# para más detalles sobre la cuadrícula de parámetros visitar:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html


Cuando estamos fijando los parámetros de la búsqueda exhaustiva, así es que indicamos los parámetros:

- preprocessor__numerical__imputer__strategy': ['mean', 'median'],

la línea de código anterior indica que queremos probar el 'imputer' con la media y la mediana en el paso de sustitución del procesador numérico 

- preprocessor__categorical__imputer__strategy': ['most_frequent', 'constant']

la línea de código anterior indica que queremos probar la sustitución por categórica más frecuente o un valor constante en el paso del sustitución del procesador para variables calóricas

- classifier__alpha': [0.1, 1.0, 0.5]

la línea de código anterior indica que queremos probar esos 3 valores del parámetro alfa para Lasso. Nota que Lasso es el paso de modelamiento o 'regresor' de nuestra tubería o pipeline


In [10]:
# ahora vamos a entrenar sobre todas las posibles combinaciones
# de los parámetros anteriores
grid_search.fit(X_train, y_train)

# imprimimos el mejor score o puntaje sobre el segmento de prueba
print(("mejor regresión linear de la busqueda exahustiva: %.3f"
       % grid_search.score(X_train, y_train)))

C:\Users\Sole\Anaconda3\envs\feml\lib\site-packages\sklearn\model_selection\_search.py:825: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


mejor regresión linear de la busqueda exahustiva: 0.933


In [11]:
# podemos imprimir los mejores parametros estimados asi

grid_search.best_estimator_

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                       

In [12]:
# los mejores parámetros ajustados asi
grid_search.best_params_

{'classifier__alpha': 100,
 'preprocessor__categorical__imputer__strategy': 'constant',
 'preprocessor__numerical__imputer__strategy': 'median'}

In [13]:
# podemos ver todas las combinaciones evaluadas durante el gridsearch
grid_search.cv_results_['params']

[{'classifier__alpha': 10,
  'preprocessor__categorical__imputer__strategy': 'most_frequent',
  'preprocessor__numerical__imputer__strategy': 'mean'},
 {'classifier__alpha': 10,
  'preprocessor__categorical__imputer__strategy': 'most_frequent',
  'preprocessor__numerical__imputer__strategy': 'median'},
 {'classifier__alpha': 10,
  'preprocessor__categorical__imputer__strategy': 'constant',
  'preprocessor__numerical__imputer__strategy': 'mean'},
 {'classifier__alpha': 10,
  'preprocessor__categorical__imputer__strategy': 'constant',
  'preprocessor__numerical__imputer__strategy': 'median'},
 {'classifier__alpha': 100,
  'preprocessor__categorical__imputer__strategy': 'most_frequent',
  'preprocessor__numerical__imputer__strategy': 'mean'},
 {'classifier__alpha': 100,
  'preprocessor__categorical__imputer__strategy': 'most_frequent',
  'preprocessor__numerical__imputer__strategy': 'median'},
 {'classifier__alpha': 100,
  'preprocessor__categorical__imputer__strategy': 'constant',
  'pre

In [14]:
# y ahora los puntajes para cada una de las combinaciones anteriores
grid_search.cv_results_['mean_test_score']

array([0.84746254, 0.84739594, 0.84814964, 0.8481309 , 0.86624908,
       0.86621021, 0.86646886, 0.86651035, 0.86552764, 0.8654755 ,
       0.86525292, 0.86523714])

In [15]:
# y finalmente miremos el desempeño en el segmento de prueba
print(("best linear regression from grid search: %.3f"
       % grid_search.score(X_test, y_test)))

best linear regression from grid search: 0.738


Este modelo sobre-ajusta el segmento de enteramiento; mirando el valor de r2 de 0.93 que obtuvimos en el segmento de entrenamiento vs. 0.738 para el segmento de prueba.

Trataremos de reducir este sobre-ajuste en las secciones más adelante del curso!